# AI/BI Dashboards & Visualization in Databricks

**Objective:** Learn how to build intelligent dashboards using Databricks AI/BI. We will move from data engineering to data consumption, creating visualizations that leverage the semantic models we built previously.

### What we will cover:
1.  **Creating AI/BI Dashboards**: Understanding the interface and SQL Warehouse requirements.
2.  **Visualizations**: Creating Counters, Pie Charts, and Bar Charts.
3.  **Calculated Fields**: defining metrics like 'Total Tax' on the fly.
4.  **AI Capabilities**:
    *   Generating visualizations using natural language prompts.
    *   **AI Forecasting** on time-series data.
    *   Enhancing AI accuracy by adding comments/metadata to Unity Catalog.
5.  **Using Metric Views**: Leveraging semantic models for consistent reporting.
6.  **Consumer Access**: Setting up read-only access for business users.

**Note:** AI/BI Dashboards are free to use; you only pay for the underlying SQL Warehouse compute.

## 1. Prerequisites & Data Setup

Ensure you have the following tables from previous lessons in your Unity Catalog (`dev.bronze`):
*   `orders_raw`
*   `customer_raw`
*   `orders_raw_metric_view` (Created in the Semantic Models lesson)

### Step 1.1: Verify Data Availability
Let's run a quick check to ensure our raw tables are populated.

In [ ]:
%sql
-- Check Orders Data
SELECT count(*) as total_orders FROM dev.bronze.orders_raw;

-- Check Customer Data
SELECT count(*) as total_customers FROM dev.bronze.customer_raw;

## 2. Improving AI Accuracy with Metadata

The "AI" in AI/BI Dashboards relies heavily on the metadata (comments) of your tables to understand what data represents. If your columns (e.g., `o_orderkey`, `o_totalprice`) don't have descriptions, the AI might struggle to generate accurate charts from natural language prompts.

We can add these comments via the UI (Catalog Explorer) or programmatically using SQL. Let's add them programmatically to ensure our AI assistant works perfectly.

In [ ]:
%sql
-- Adding comments to 'orders_raw' to help the AI Assistant
ALTER TABLE dev.bronze.orders_raw ALTER COLUMN o_orderkey COMMENT 'Unique identifier for the order';
ALTER TABLE dev.bronze.orders_raw ALTER COLUMN o_custkey COMMENT 'Identifier for the customer who placed the order';
ALTER TABLE dev.bronze.orders_raw ALTER COLUMN o_totalprice COMMENT 'Total price of the order';
ALTER TABLE dev.bronze.orders_raw ALTER COLUMN o_orderdate COMMENT 'Date when the order was placed';
ALTER TABLE dev.bronze.orders_raw ALTER COLUMN o_orderstatus COMMENT 'Status of the order: O=Open, F=Fulfilled, P=Pending';

-- Adding comments to 'customer_raw'
ALTER TABLE dev.bronze.customer_raw ALTER COLUMN c_custkey COMMENT 'Unique identifier for the customer';
ALTER TABLE dev.bronze.customer_raw ALTER COLUMN c_mktsegment COMMENT 'Market segment of the customer (e.g., AUTOMOBILE, MACHINERY)';
ALTER TABLE dev.bronze.customer_raw ALTER COLUMN c_name COMMENT 'Name of the customer';

## 3. Creating the Dashboard (UI Steps)

1.  Navigate to **Dashboards** in the sidebar (or click **New > Dashboard**).
2.  **Name the Dashboard**: e.g., "Sales Dashboard".
3.  **Select Warehouse**: Attach a SQL Warehouse (Serverless is recommended for faster startup).

### A. Adding Data
You will see a **Data** tab on the left. We need to add our datasets.
1.  Click **Add data source**.
2.  Select `dev.bronze.orders_raw`.
3.  Select `dev.bronze.orders_raw_metric_view`.
4.  (Optional) Create a **Joined Dataset** via SQL (Instructions below).

### B. Creating a Joined Dataset via SQL
In the Data tab, click **Create from SQL** and paste the following logic to combine Orders and Customers. This allows us to slice orders by market segment.

In [ ]:
-- You can copy this query into the Dashboard SQL Editor to create a dataset named 'Customer_Orders_Joined'
SELECT 
    o.o_orderkey,
    o.o_totalprice,
    o.o_orderdate,
    o.o_orderstatus,
    c.c_mktsegment,
    c.c_name
FROM dev.bronze.orders_raw o
LEFT OUTER JOIN dev.bronze.customer_raw c
ON o.o_custkey = c.c_custkey

## 4. Building Visualizations

We will create the following widgets on the canvas:

### Visualization 1: Total Orders (Counter)
*   **Dataset:** `orders_raw`
*   **Visualization Type:** Counter
*   **Value:** `o_orderkey` -> Aggregation: **Count Distinct**
*   **Title:** "Total Orders"
*   **Formatting:** Select "Compact" number formatting (e.g., 7.5M).

### Visualization 2: Total Sales (Counter)
*   **Dataset:** `orders_raw`
*   **Value:** `o_totalprice` -> Aggregation: **Sum**
*   **Title:** "Total Sales"
*   **Formatting:** Currency (USD), Compact.

### Visualization 3: Order Status Distribution (Pie Chart)
*   **Dataset:** `orders_raw`
*   **Angle:** `o_orderkey` (Count Distinct)
*   **Color:** `o_orderstatus`
*   **Interaction:** Clicking a slice (e.g., "F") will filter other charts on the dashboard.

### Visualization 4: AI Generated Chart
*   Click the "Assistant" bar on a new widget.
*   **Prompt:** "Show me order status distribution"
*   The AI will generate the chart automatically based on the comments we added in step 2.

## 5. Advanced Features: Custom Calculations & Forecasting

### Custom Calculated Fields
Sometimes data is missing from the raw table (e.g., Tax). We can define it in the dashboard without changing the underlying DDL.
1.  In the **Data** tab, click on `orders_raw`.
2.  Click **Custom Calculation** (Calculator icon).
3.  **Name:** `Total Tax`
4.  **Formula:** `SUM(o_totalprice) * 0.10` (Assuming 10% tax).
5.  Use this new field in a Counter visualization.

### AI Forecasting
Let's predict future orders based on historical data.
1.  Create a **Line Chart**.
2.  **X-Axis:** `o_orderdate` (Transform: Yearly).
3.  **Y-Axis:** `o_orderkey` (Count Distinct).
4.  **Action:** In the configuration panel, toggle **Forecast (Beta)**.
5.  **Result:** Databricks uses Prophet/ARIMA models to project the line forward with confidence intervals.

## 6. Using Metric Views (Semantic Models)

Instead of redefining `Count Distinct` and `Sum` every time, we use the **Metric View** created in the previous video.

1.  Select dataset: `orders_raw_metric_view`.
2.  Create a **Bar Chart**.
3.  **X-Axis:** `Order Year` (Dimension).
4.  **Y-Axis:** `Total Orders` (Measure).
5.  **Benefit:** The logic for "Total Orders" or "Year over Year Growth" is centrally managed. If the business logic changes, you update the metric view, not 50 different dashboards.

## 7. Publishing & Consumer Access

### Publishing
Click the **Publish** button. You can choose to embed credentials (viewers use your access) or not (viewers use their own access).

### Setting up Business User Access (Consumer Profile)
For users who **only** need to view dashboards and not run code/notebooks:
1.  Go to **Account Console** -> **User Management**.
2.  Create a Group: `business_user_grp`.
3.  **Entitlements:** Assign **"Consumer Access"** (This removes Workspace/SQL Editor access).
4.  **Workspace Access:** Grant the group access to the specific workspace.
5.  **Dashboard Sharing:** In the dashboard, click **Share**, add `business_user_grp`, and grant **"Can Run"** permissions.

*This creates a clean, clutter-free environment for executives/business users.*